In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook

In [2]:
# links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
# tags = pd.read_csv('tags.csv')

In [3]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [5]:
# соединим таблицу с рейтингами и названиями фильмов
joined_ratings = ratings.join(movies.set_index('movieId'), on='movieId')

In [6]:
joined_ratings.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [9]:
# достанем по каждому фильму количество рейтингов
title_num_ratings = {}

for title, group in tqdm_notebook(joined_ratings.groupby('title')):
    title_num_ratings[title] = group.userId.unique().shape[0]

In [10]:
min_num_ratings = np.min(list(title_num_ratings.values()))
max_num_ratings = np.max(list(title_num_ratings.values()))
# mean_num_ratings = np.mean(list(title_num_ratings.values()))
# median_num_ratings = np.median(list(title_num_ratings.values()))

In [11]:
def top_rate(df, min_num, max_num): #минималное и максимальное кол-во рейтингов передаем для равных весов оценок
    # достанем по каждому фильму количество рейтингов
    title_num_ratings = {}
    title_mean_rating = {}

    for title, group in df.groupby('title'):
        title_num_ratings[title] = group.userId.unique().shape[0]
        title_mean_rating[title] = group.rating.mean()
        
    film_with_our_mark = {}

    # посчитаем нашу метрику для каждого фильма из датасета
    for f in title_num_ratings.keys():
        film_with_our_mark[f] = title_mean_rating[f] * \
        (title_num_ratings[f] - min_num_ratings)/ (max_num_ratings - min_num_ratings)
    
    return sorted(film_with_our_mark.items(), key=lambda x: x[1], reverse=True)[:10]

In [12]:
comedy_df = joined_ratings[joined_ratings.genres.str.find('Comedy')>=0]

In [13]:
top_rate(comedy_df, min_num_ratings, max_num_ratings)

[('Forrest Gump (1994)', 4.164133738601824),
 ('Pulp Fiction (1994)', 3.9155577182807666),
 ('Toy Story (1995)', 2.5581678956324447),
 ('Fargo (1996)', 2.2587926155504645),
 ('Aladdin (1992)', 2.104291616686659),
 ('Back to the Future (1985)', 2.0928719155612607),
 ('Shrek (2001)', 1.9927815638450501),
 ('True Lies (1994)', 1.887203685941354),
 ('Princess Bride, The (1987)', 1.8194134318103743),
 ('Men in Black (a.k.a. MIB) (1997)', 1.743939393939394)]

In [14]:
action_df = joined_ratings[joined_ratings.genres.str.find('Action')>=0]

In [15]:
top_rate(action_df, min_num_ratings, max_num_ratings)

[('Matrix, The (1999)', 3.5405718108440083),
 ('Star Wars: Episode IV - A New Hope (1977)', 3.2249052570206973),
 ('Braveheart (1995)', 2.900818153751158),
 ('Fight Club (1999)', 2.826911781159096),
 ('Jurassic Park (1993)', 2.7096036585365852),
 ('Terminator 2: Judgment Day (1991)', 2.6997835910278747),
 ('Star Wars: Episode V - The Empire Strikes Back (1980)', 2.6990376835047973),
 ('Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)',
  2.5527210365853654),
 ('Star Wars: Episode VI - Return of the Jedi (1983)', 2.4599458685913387),
 ('Saving Private Ryan (1998)', 2.363883303061754)]